# FFF Workshop

## A6: Uploading to Fragalysis

### Outline

- Export Poses as Fragalysis compatible SD files (SDFs)
- Upload SDFs to the Fragalysis RHS
- Upload interactive Plotly Figures to the RHS

## Export Poses as Fragalysis compatible SD files

In [ ]:
# setup the animal
%load_ext autoreload
%autoreload 2
import hippo
animal = hippo.HIPPO(
    "A71EV2A_demo",
    "../data/A71EV2A.sqlite",
)

### Filter poses

Poses can be filtered by their Fragmenstein assigned *energy_score* (ddG) and *distance_score* (RMSD):

In [ ]:
filtered_merges = (
    animal.poses(
        tag="openbind_a71ev2a_c1_scaffolds",
    ).filter(
        key="energy_score", 
        value="-10", 
        operator="<",
    ).filter(
        key="distance_score", 
        value="0.75", 
        operator="<",
    )
)

filtered_merges

## Upload SDF to Fragalysis RHS

Uploading SDFs to Fragalysis currently has to be done via the browser using the `viewer/upload_cset` endpoint. E.g. for staging:

https://fragalysis.xchem.diamond.ac.uk/viewer/upload_cset/

See the documentation here: https://fragalysis.readthedocs.io/en/latest/rhs_upload.html

You can try this with the `../data/openbind_a71ev2a_c1_scaffolds.sdf` or create your own Fragalysis compatible subset using the cells below:

In [ ]:
selected_poses = animal.poses["A4350a"].derivatives

selected_poses.to_fragalysis(
    "A4350a_derivatives.sdf",
    method="A4350a_derivatives",
    submitter_name="YOUR NAME",
    submitter_institution="INSTITUTION",
    submitter_email="EMAIL",
)

## Upload Plotly Figure to RHS

Fragalysis now supports the upload of arbitrary interactive graphs to the RHS. This can be used to provide additional statistical or analytic insight or serve as an alternative way to navigate observations.

Some HIPPO graphs come pre-formatted to support Fragalysis out the box. In general, if a HIPPO graph has compound codes or observation names in it's x-axis it is likely to work.

First, create a figure object:

In [ ]:
fig = animal.plot_residue_interactions(107)
fig

Make any formatting changes:

In [ ]:
fig.update_layout(xaxis_title="Observation");

Send the upload request:

In [ ]:
from os import environ
from fragalysis.requests import upload_graph
upload_graph(
    title="Interactions w/ PRO 107",
    figure=fig,
    stack="staging",
    token=environ["FRAGALYSIS_TOKEN"],
    target_name="A71EV2A",
    tas="lb32627-66",
)

N.B. that if on-click events are to reference compounds, `identifier="compound_code"` should be added to the above function call

Plots can be deleted again by navigating to `/api/plot_data/<plot_id>` in Fragalysis